# Transactions 2003-2016
This notebook will explore transactional data between 2003 and 2016.  This data has been distilled down to show the expenses and revenue on a per vendor (using the term vendor loosely, because payroll is being included in vendor).  The data has been loaded onto a mainframe as physical sequential files and will be processed using a jupyter notebook running on the same LPAR.

## Import a few popular python packages and dsdbc

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

#Remove font warnings
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore", category=PendingDeprecationWarning)

***Note: dsdbc is a the python driver for the Optimized Data Virtualization Layer of the IBM Open Data Analytics on z product.***

In [ ]:
# To use dsdbc, if you have the necessary files virtualized into ODL, simply comment out the next line
csv = "yes"

if not csv:
    import dsdbc #This package required to interface with ODL
    conn = dsdbc.connect()
    cursor = conn.cursor()

# Load the Transaction Data
Load the expense transactions and the revenue transactions from physical sequential files into Panda Dataframes

In [ ]:
#Create pandas dataframe directly from "DB" query (really a pysical sequential dataset)
if csv:
    exp_df = pd.read_csv("data/VI_03_16_EXP_VIEW2.csv")
    rev_df = pd.read_csv("data/VI_03_16_REV_VIEW2.csv")
else:
    exp_df = pd.read_sql("SELECT * FROM VI_03_16_EXP_VIEW2", conn)
    rev_df = pd.read_sql("SELECT * FROM VI_03_16_REV_VIEW2", conn)

In [ ]:
exp_df

## Compare Yearly Expenses and Revenue
Calculate the yearly totals of expenses and Revenue, then see what they net.

In [ ]:
trans_dict = {
    2003: [],
    2004: [],
    2005: [],
    2006: [],
    2007: [],
    2008: [],
    2009: [],
    2010: [],
    2011: [],
    2012: [],
    2013: [],
    2014: [],    
    2015: [],
    2016: [],
}
trans_list = []
years_df = pd.DataFrame()
for year in trans_dict:
    trans_list = [str(year),\
                  exp_df['Y' + str(year)].astype(float).sum(),\
                  rev_df['Y' + str(year)].astype(float).sum(),\
                  (rev_df['Y' + str(year)].astype(float).sum() - exp_df['Y' + str(year)].astype(float).sum())
                 ]
    if (years_df.empty):
        years_df = pd.DataFrame([trans_list], columns=["YEAR", 'EXP', 'REV', 'NET'])
    else:
        years_df = years_df.append(pd.DataFrame([trans_list], columns=["YEAR", 'EXP', 'REV', 'NET']),ignore_index=True) 
years_df 

### Graph the yearly results

In [ ]:
sns.barplot(x="YEAR", y="EXP", data=years_df)
sns.plt.show()

In [ ]:
sns.barplot(x="YEAR", y="REV", data=years_df)
sns.plt.show()

In [ ]:
sns.barplot(x="YEAR", y="NET", data=years_df)
sns.plt.show()

## Vendor's Per Year

### Load the Vendor Data
The vendor data is in a physical sequential file and is being loaded using dsbdc

In [ ]:
#Create pandas dataframe directly from "DB" query (really a pysical sequential dataset)
if csv:
    vendor_df = pd.read_csv("data/SAFR_VEND_L-1.csv")
    vendor_df2 = pd.read_csv("data/SAFR_VEND_L-2.csv")
    vendor_df = vendor_df.append(vendor_df2)
    vendor_df3 = pd.read_csv("data/SAFR_VEND_L-3.csv")
    vendor_df =vendor_df.append(vendor_df3)  
else:
    vendor_df = pd.read_sql("SELECT VVR_INST_ID, VVR_HOLDER_NAME FROM SAFR_VEND_L", conn)
vendor_df['VVR_INST_ID'] = vendor_df['VVR_INST_ID'].astype("int64")
vendor_df

Merge the Vendor to the transactional dataframes

In [ ]:
exp_df = exp_df.merge(vendor_df, right_on="VVR_INST_ID", left_on="INST_ID", how="left")
rev_df = rev_df.merge(vendor_df, right_on="VVR_INST_ID", left_on="INST_ID", how="left")
exp_df

Calculate yearly total per vendor (both expense and revenue)

In [ ]:
exp_df["Total"] = exp_df["Y2003"] + exp_df["Y2004"] + exp_df["Y2005"] + exp_df["Y2006"] + exp_df["Y2007"] + \
exp_df["Y2008"] + exp_df["Y2009"] + exp_df["Y2010"] + exp_df["Y2011"] + exp_df["Y2012"] + exp_df["Y2013"] + \
exp_df["Y2014"] + exp_df["Y2015"] + exp_df["Y2016"]
rev_df["Total"] = rev_df["Y2003"] + rev_df["Y2004"] + rev_df["Y2005"] + rev_df["Y2006"] + rev_df["Y2007"] + \
rev_df["Y2008"] + rev_df["Y2009"] + rev_df["Y2010"] + rev_df["Y2011"] + rev_df["Y2012"] + rev_df["Y2013"] + \
rev_df["Y2014"] + rev_df["Y2015"] + rev_df["Y2016"]

Calculate the averages per year, per vendor (both expense and revenue)

In [ ]:
num_of_years = (2016 - 2003) + 1
exp_df["Average"] = exp_df["Total"].astype(float) / num_of_years
rev_df["Average"] = rev_df["Total"].astype(float) / num_of_years

## Top 20% of Vendors
What percentages of the overal transactions are these Vendors responisble for

In [ ]:
# Calculate absolute value of all expense transactions
pos_exp_df = exp_df.loc[exp_df['Total'] > 0]
neg_exp_df = exp_df.loc[exp_df['Total'] < 0]
neg_exp_df['Total'] = neg_exp_df['Total'] * -1
pos_total_exp = pos_exp_df['Total'].astype(float).sum()
neg_total_exp = neg_exp_df['Total'].astype(float).sum()
grand_total_exp = pos_total_exp + neg_total_exp
print("${:,.2f}".format(grand_total_exp))

In [ ]:
# Calculate absolute value of all revenue transactions
pos_rev_df = rev_df.loc[rev_df['Total'] > 0]
neg_rev_df = rev_df.loc[rev_df['Total'] < 0]
neg_rev_df['Total'] = neg_rev_df['Total'] * -1
pos_total_rev = pos_rev_df['Total'].astype(float).sum()
neg_total_rev = neg_rev_df['Total'].astype(float).sum()
grand_total_rev = pos_total_rev + neg_total_rev
print("${:,.2f}".format(grand_total_rev))

In [ ]:
# Top 20% of vendors
top_20_percent_exp_df = exp_df.nlargest(round(len(exp_df.index)*.2), 'Total')
top_20_total_exp = top_20_percent_exp_df['Total'].astype(float).sum()
print("${:,.2f}".format(top_20_total_exp))

In [ ]:
# Top 20% of vendors
top_20_percent_rev_df = rev_df.nlargest(round(len(rev_df.index)*.2), 'Total')
top_20_total_rev = top_20_percent_rev_df['Total'].astype(float).sum()
print("${:,.2f}".format(top_20_total_rev))

In [ ]:
complete_df = exp_df[['INST_ID','Total']].append(rev_df[['INST_ID','Total']])
top_20_df = complete_df.nlargest(round(len(complete_df.index)*.2), 'Total')
top_20_total = top_20_percent_rev_df['Total'].astype(float).sum()
print("${:,.2f}".format(top_20_total))

In [ ]:
grand_total = grand_total_exp + grand_total_rev
print("${:,.2f}".format(grand_total))

***What is the actual percentage?***

In [ ]:
print ("{0:.0f}%".format(top_20_total / grand_total * 100))

In [ ]:
fracs = [100-(top_20_total / grand_total * 100), (top_20_total / grand_total * 100)]
labels = ["Total", "Top 20%"]
plt.pie(fracs, explode=(0,0.2), labels=labels, autopct='%1.1f%%', shadow=True)
plt.show()

## Top 5 Expense Vendors

In [ ]:
# Top 5 vendors
top_5_vendor_exp_df = exp_df.nlargest(5, 'Total')
ids = top_5_vendor_exp_df['INST_ID']
top_5_vendor_exp_df

In [ ]:
a4_dims = (5, 3)
fig, ax = plt.subplots(figsize=a4_dims)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
sns.barplot(x="VVR_HOLDER_NAME", y="Total", data=top_5_vendor_exp_df)
sns.plt.show()

In [ ]:
a4_dims = (15, 5)
fig, ax = plt.subplots(figsize=a4_dims)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)

for key in ids:
    df = top_5_vendor_exp_df[top_5_vendor_exp_df["VVR_INST_ID"] == key]
    vendor_name = df['VVR_HOLDER_NAME']
    df = df.squeeze().to_frame()
    df = df.drop(['RECNUM', 'INST_ID', 'VVR_INST_ID', 'VVR_HOLDER_NAME', 'Total', 'Average'])
    df = df.reset_index()
    df.columns = ['Year', 'Total']
    df['Year'] = df['Year'].str[1:]
    plt.plot(df)
    
plt.xlabel('Year', fontsize=14, color='red')
plt.ylabel('Total', fontsize=14, color='red')
sns.plt.show()

In [ ]:
for key in ids:
    df = top_5_vendor_exp_df[top_5_vendor_exp_df["VVR_INST_ID"] == key]
    vendor_name = df['VVR_HOLDER_NAME']
    df = df.squeeze().to_frame()
    df = df.drop(['RECNUM', 'INST_ID', 'VVR_INST_ID', 'VVR_HOLDER_NAME', 'Total', 'Average'])
    df = df.reset_index()
    df.columns = ['Year', 'Total']
    df['Year'] = df['Year'].str[1:]
    sns.barplot(x="Year", y="Total", data=df)
    print(vendor_name)
    sns.plt.show()

## Top 5 Revenue Vendors

In [ ]:
# Top 5 vendors
top_5_vendor_rev_df = rev_df.nlargest(5, 'Total')
ids = top_5_vendor_rev_df['INST_ID']
top_5_vendor_rev_df

In [ ]:
a4_dims = (5, 3)
fig, ax = plt.subplots(figsize=a4_dims)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
sns.barplot(x="VVR_HOLDER_NAME", y="Total", data=top_5_vendor_rev_df)
sns.plt.show()

In [ ]:
a4_dims = (15, 5)
fig, ax = plt.subplots(figsize=a4_dims)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)

for key in ids:
    df = top_5_vendor_rev_df[top_5_vendor_rev_df["VVR_INST_ID"] == key]
    vendor_name = df['VVR_HOLDER_NAME']
    df = df.squeeze().to_frame()
    df = df.drop(['RECNUM', 'INST_ID', 'VVR_INST_ID', 'VVR_HOLDER_NAME', 'Total', 'Average'])
    df = df.reset_index()
    df.columns = ['Year', 'Total']
    df['Year'] = df['Year'].str[1:]
    plt.plot(df)
    
plt.xlabel('Year', fontsize=14, color='red')
plt.ylabel('Total', fontsize=14, color='red')
sns.plt.show()

In [ ]:
for key in ids:
    df = top_5_vendor_rev_df[top_5_vendor_rev_df["VVR_INST_ID"] == key]
    vendor_name = df['VVR_HOLDER_NAME']
    df = df.squeeze().to_frame()
    df = df.drop(['RECNUM', 'INST_ID', 'VVR_INST_ID', 'VVR_HOLDER_NAME', 'Total', 'Average'])
    df = df.reset_index()
    df.columns = ['Year', 'Total']
    df['Year'] = df['Year'].str[1:]
    sns.barplot(x="Year", y="Total", data=df)
    print(vendor_name)
    sns.plt.show()